# Contrast Grid

<div style="text-align: justify">
Applefy provides two methods for calculating detection limits: **Analytical Contrast Curves** and **Contrast Grids**.

This tutorial focuses on contrast grids, which are more computationally demanding, but yield more intuitive and reliable results. In addition, they are not limited to being used with linear PSF subtraction techniques (such as PCA).

Contrast Grids indicate whether a planet would be detected based on its planet-to-star flux ratio and separation from the host star. All tests described in [the statistics subpackage](../../03_package/statistics.rst#applefy.statistics) can be used for the calculation. The result of a contrast grid **can be transformed into contrast curves** through thresholding.

</div>

<div style="text-align: justify">
The following plot gives an example of a contrast grid computed with applefy. The image on the left shows a residual image obtained with PCA showing a fake planet. On the right the actual contrast grid is shown. The red box in the contrast grid marks the brightness and separation of the inserted fake planet. The color of the contrast grid is the detection uncertatinty (FPF) in terms of $\sigma_{N}$. Planets with $> 5 \sigma_{N}$ are easy to see by eye. Planets with less than $3 \sigma_{N}$ are not detectable.

<font color='red'>Click on the GIF animation to load the interactive version of the plot (~70 MB)</font>. 
</div>

In [1]:
from IPython.display import HTML
HTML('contrast_grid_animation.html')

## Imports

In [2]:
from pathlib import Path

import seaborn as sns
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from applefy.detections.contrast import Contrast

from applefy.utils.photometry import AperturePhotometryMode
from applefy.statistics import TTest, gaussian_sigma_2_fpf, \
    fpf_2_gaussian_sigma, LaplaceBootstrapTest

from applefy.utils.file_handling import load_adi_data
from applefy.utils import flux_ratio2mag, mag2flux_ratio

## Demonstration Data

The following tutorial shows how to compute contrast grids. The data used can be download [from Zenodo](../01_getting_started.md).

In [3]:
root_dir = Path("/home/ipa/quanz/user_accounts/mbonse/2021_Metrics/70_results/documentation_demos")

In [4]:
dataset_config = {'file_path': Path('30_data/betapic_naco_lp_stack.hdf5'),
                  'stack_key': "stack_no_planet",
                  'psf_template_key': "flux",
                  'parang_key': 'header_stack/PARANG',
                  'dit_psf_template': 0.02019,
                  'fwhm_size' : 4.2,
                  'dit_science': 0.2}

In general we need three data arrays to compute contrast grids:
- The **science sequence**: A 3D array with dimensions (time, x, y)
- A list of **parallactic angles** for Angular Differentail Imaging (time)
- A 2D **PSF-template** which is used to insert artificial planets and determine the detection limits.

We load them from the demonstration  data. In order to save computation time we reduce the resolution of the science sequence by cutting around the central star.

In [5]:
# we need the psf template for contrast calculation
science_data, angles, raw_psf_template_data = load_adi_data(
    root_dir/dataset_config["file_path"],
    data_tag=dataset_config["stack_key"],
    psf_template_tag=dataset_config["psf_template_key"],
    para_tag=dataset_config["parang_key"])

dit_psf_template = dataset_config["dit_psf_template"]
dit_science = dataset_config["dit_science"]
fwhm = dataset_config["fwhm_size"]

psf_template = raw_psf_template_data[82:-82, 82:-82]
science_data = science_data[:, 55:-55, 55:-55]

## How to compute a Contrast Grid?

The main interface to calculate analytical contrast curves as well as contrast grids is the class [Contrast](../03_package/detections.rst#applefy.applefy.detections.contrast.Contrast). It takes the science sequence, parallactic angles and the PSF-template needed to run fake planet experiments. 

It further needs the FWHM of the PSF (can be calculated e.g. with [PynPoint](https://pynpoint.readthedocs.io/en/latest/) or
[VIP](https://vip.readthedocs.io/en/latest/>)) as well as the DIT of both science sequence and PSF-template. 

A checkpoint directroy can be given to store intermediate results (such as residuals). Such a directroy is optional but allows to restore fake planet residuals at a later stage. More information can be found in the [API documnetation](../03_package/detections.rst#applefy.applefy.detections.contrast.Contrast).

In [6]:
contrast_instance = Contrast(
    science_sequence=science_data,
    psf_template=psf_template,
    parang_rad=angles,
    psf_fwhm_radius=fwhm / 2,
    dit_psf_template=dit_psf_template,
    dit_science=dit_science,
    checkpoint_dir=root_dir / Path("70_results/contrast_grid"))

### Step 1: Design fake planet experiments

The first step is to **choose** at which **separations** and for which **planet brightnesses** we want to insert fake planets. By default, separations are selected in steps of 1 FWHM form the central star to the edge of the image. The brightness of the fake planets as to be selected manually ([more information](../03_package/detections.rst#applefy.applefy.detections.contrast.Contrast.design_fake_planet_experiments)).


Note: The resolution we choose in this step will **determine the grid resolution of the contrast grid**.

In [7]:
# fake planet brightness
flux_ratios_mag = np.linspace(7.5, 12, 10)
flux_ratios = mag2flux_ratio(flux_ratios_mag)

print("Brightness of fake planets in mag: " + str(flux_ratios_mag))
print("Planet-to-star flux ratio: " + str(flux_ratios))

Brightness of fake planets in mag: [ 7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12. ]
Planet-to-star flux ratio: [1.00000000e-03 6.30957344e-04 3.98107171e-04 2.51188643e-04
 1.58489319e-04 1.00000000e-04 6.30957344e-05 3.98107171e-05
 2.51188643e-05 1.58489319e-05]


For each cell in the contrast grid we calculate *num_fake_planets* (between min=1 and max=6) fake planet residuals.

In [8]:
num_fake_planets = 3

contrast_instance.design_fake_planet_experiments(
    flux_ratios=flux_ratios,
    num_planets=num_fake_planets,
    overwrite=True)

Overwriting existing config files.


## Step 2: Run fake planet experiments

As a next step we **insert fake planetes** and **run data reduction** algorithms to obtain several residual images. Applefy allows to compute contrast grids with different post-processing algorithms. However, it does not come with any implementation of these techniques. Instead, we use existing implementations in packages like
[PynPoint](https://pynpoint.readthedocs.io/en/latest/) or
[VIP](https://vip.readthedocs.io/en/latest/>). See [wrappers](../03_package/wrappers.rst) for more information.

In the following we use a **classical full-frame PCA** to compute the residuals. But applefy can be used with any other data-reduction algorithm.

Typically, detection limits depend on the choice of post-processing hyper-parameters (e.g. the number of PCA components). Applefy allows you to compute contrast grids for several hyperparameter choices simultaneously.

Here is an example of how to use the wrapper to run fake planet experiments **with PynPoint** (requires a checkpoint_dir).

In [9]:
from applefy.wrappers.pynpoint import MultiComponentPCAPynPoint

In [10]:
components = [5, 10, 20, 30, 50]

In [11]:
algorithm_function = MultiComponentPCAPynPoint(
    num_pcas=components,
    scratch_dir=contrast_instance.scratch_dir,
    num_cpus_pynpoint=1)

Here is an example of how to use the wrapper to run fake planet experiments **with VIP**.

In [ ]:
from applefy.wrappers.vip import MultiComponentPCAvip

In [21]:
algorithm_function = MultiComponentPCAvip(
    num_pcas_tuple=(5, 50, 10))

Applefy can run multiple fake planet experiments in parallel. If a checkpoint_dir is specified, all residuals will be saved as .fits files. Already existing residuals are restored from the checkpoint_dir.

In [12]:
contrast_instance.run_fake_planet_experiments(
    algorithm_function=algorithm_function,
    num_parallel=32)

Running fake planet experiments...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 241/241 [00:05<00:00, 43.34it/s]


[DONE]


## Step 3: Compute the contrast grid

Once all fake planet experiments are finished we can **compute the contrast grid**. For this we have to choose:
- What is signal and what is noise, i.e. **how we measure the photometry** of the noise observations and the signal of the fake planets. Applefy allows to use aperture sums and spaced pixel values. A discussion about when to use what is given in (Bonse et al. 2023). More details are given in the API [documentation](../03_package/utils.rst#applefy.utils.photometry.AperturePhotometryMode).

In [16]:
# Use spaced pixel values
photometry_mode_planet = AperturePhotometryMode(
    "FS", # or "P"
    psf_fwhm_radius=fwhm/2, 
    search_area=0.5)
photometry_mode_noise = AperturePhotometryMode(
    "P", 
    psf_fwhm_radius=fwhm/2)

In [15]:
# Use apertures pixel values
photometry_mode_planet = AperturePhotometryMode(
    "AS", # or "ASS"
    psf_fwhm_radius=fwhm/2, 
    search_area=0.5)

photometry_mode_noise = AperturePhotometryMode(
    "AS",
    psf_fwhm_radius=fwhm/2)

In [14]:
contrast_instance.prepare_contrast_results(
    photometry_mode_planet=photometry_mode_planet,
    photometry_mode_noise=photometry_mode_noise)

- Which statistical test we want to use i.e. what type of noise we assume for our residual images.

**Option 1:** The classical [t-test](../03_package/statistics.rst#applefy.statistics.parametric.TTest) as discussed in Mawet et al. 2014 (assumes **Gaussian residual noise**).

In [17]:
statistical_test = TTest()

**Option 2:** The [Parametric Bootstrap](../03_package/statistics.rst#applefy.statistics.bootstrapping.LaplaceBootstrapTest) test as discussed in Bonse et al. 2023 (assumes **Laplacian residual noise**). You can download the lookup [from Zenodo](../01_getting_started.md).

In [ ]:
statistical_test = LaplaceBootstrapTest.construct_from_json_file("file/to/lookup_table")

The contrast grid can again be calculated using multiprocessing.

In [ ]:
contrast_curves_grid, contrast_grids = contrast_instance.compute_contrast_grids(
    statistical_test=statistical_test,
    num_cores=32,
    confidence_level_fpf=gaussian_sigma_2_fpf(5),
    num_rot_iter=20,
    safety_margin=2.0,
    pixel_scale=0.02718)

Computing contrast grid for PCA (005 components)
Computing contrast grid with multiprocessing:
................................................................................[DONE]
Computing contrast grid for PCA (010 components)
Computing contrast grid with multiprocessing:
................

We get two outputs:
- A dict which contains the contrast grids (one for each number of PCA components).
- A Pandas table showing the contrast curves obtained by thresholding the contrast grids.

In [ ]:
contrast_curves_grid

## Plot the Contrast Grid

The following code gives an example for how to plot a contrast grid.

In [ ]:
example_grid = contrast_grids["PCA (030 components)"]

# convert FPF to Gaussian Sigma
example_grid = example_grid.applymap(fpf_2_gaussian_sigma) 

# convert flux_ratio to mag
example_grid.index = flux_ratio2mag(example_grid.index)
example_grid

In [ ]:
def plot_contrast_grid(
    contrast_grid_axis,
    colorbar_axis,
    contrast_grid):
    
    c_bar_kargs = dict(
        orientation = "vertical",
        label = r"Confidence [$\sigma_{\mathcal{N}}$]")
    
    heat = sns.heatmap(
        contrast_grid,
        vmax=2, vmin=7, 
        annot=True,
        cmap="YlGnBu",
        ax=contrast_grid_axis,
        cbar_ax=colorbar_axis,
        cbar_kws=c_bar_kargs)
    
    ylabels = ['{:.1f}'.format(float(x.get_text())) 
               for x in heat.get_yticklabels()]
    _=heat.set_yticklabels(ylabels)

In [ ]:
fig = plt.figure(figsize=(8, 4))

gs0 = fig.add_gridspec(1, 1)
gs0.update(wspace=0.0, hspace=0.2)
gs1 = gridspec.GridSpecFromSubplotSpec(
    1, 2, subplot_spec = gs0[0], 
    wspace=0.05, width_ratios=[1, 0.03])

# All axis we need
contrast_ax = fig.add_subplot(gs1[0])
colorbar_ax = fig.add_subplot(gs1[1])

# Plot the contrast grid
plot_contrast_grid(
    contrast_grid_axis=contrast_ax,
    colorbar_axis=colorbar_ax,
    contrast_grid=example_grid)

colorbar_ax.yaxis.label.set_size(14)

contrast_ax.set_ylabel(
    "Contrast - $c = f_p / f_*$ - [mag]", size=14)
contrast_ax.set_xlabel(
    r"Separation [FWHM]", size=14)
contrast_ax.set_title(
    "Contrast Grid with 30 PCA components", 
    fontsize=16, 
    fontweight="bold", 
    y=1.03)

contrast_ax.tick_params(
    axis='both', 
    which='major', 
    labelsize=12)

# Save the figure
fig.patch.set_facecolor('white')

## Plot the contrast curves

The following code shows how we can plot and compare the contrast curves for different numbers of PCA components.

In [ ]:
# compute the overall best contrast curve
overall_best = np.min(contrast_curves_grid.values, axis=1)

In [ ]:
# Find one color for each number of PCA components used.
colors = sns.color_palette(
    "rocket_r", 
    n_colors=len(contrast_curves_grid.columns))
colors

In [ ]:
separations_arcsec = contrast_curves_grid.reset_index(level=0).index
separations_FWHM = contrast_curves_grid.reset_index(level=1).index

In [ ]:
# 1.) Create Plot Layout
fig = plt.figure(constrained_layout=False, figsize=(12, 8))
gs0 = fig.add_gridspec(1, 1)
axis_contrast_curvse = fig.add_subplot(gs0[0, 0])


# ---------------------- Create the Plot --------------------
i = 0 # color picker

for tmp_model in contrast_curves_grid.columns:
    
    num_components = int(tmp_model[6:9])
    tmp_flux_ratios = contrast_curves_grid.reset_index(
        level=0)[tmp_model].values
    
    axis_contrast_curvse.plot(separations_arcsec,
                              tmp_flux_ratios,
                              color = colors[i],
                              label=num_components)
    i+=1
    
axis_contrast_curvse.set_yscale("log")
# ------------ Plot the overall best -------------------------
axis_contrast_curvse.plot(
    separations_arcsec,
    overall_best,
    color = "blue",
    lw=3,
    ls="--",
    label="Best")

# ------------- Double axis and limits -----------------------
lim_mag_y = (11, 8)
lim_arcsec_x = (0.3, 0.95)
sep_lambda_arcse = interpolate.interp1d(
    separations_arcsec, 
    separations_FWHM, 
    fill_value='extrapolate')

axis_contrast_curvse_mag = axis_contrast_curvse.twinx()
axis_contrast_curvse_mag.plot(
    separations_arcsec,
    flux_ratio2mag(tmp_flux_ratios),
    alpha=0.)
axis_contrast_curvse_mag.invert_yaxis()

axis_contrast_curvse_lambda = axis_contrast_curvse.twiny()
axis_contrast_curvse_lambda.plot(
    separations_FWHM,
    tmp_flux_ratios,
    alpha=0.)

axis_contrast_curvse.grid(which='both')
axis_contrast_curvse_mag.set_ylim(*lim_mag_y)
axis_contrast_curvse.set_ylim(
    mag2flux_ratio(lim_mag_y[0]), 
    mag2flux_ratio(lim_mag_y[1]))

axis_contrast_curvse.set_xlim(
    *lim_arcsec_x)
axis_contrast_curvse_mag.set_xlim(
    *lim_arcsec_x)
axis_contrast_curvse_lambda.set_xlim(
    *sep_lambda_arcse(lim_arcsec_x))

# ----------- Labels and fontsizes --------------------------

axis_contrast_curvse.set_xlabel(
    r"Separation [arcsec]", size=16)
axis_contrast_curvse_lambda.set_xlabel(
    r"Separation [FWHM]", size=16)

axis_contrast_curvse.set_ylabel(
    r"Planet-to-star flux ratio", size=16)
axis_contrast_curvse_mag.set_ylabel(
    r"$\Delta$ Magnitude", size=16)

axis_contrast_curvse.tick_params(
    axis='both', which='major', labelsize=14)
axis_contrast_curvse_lambda.tick_params(
    axis='both', which='major', labelsize=14)
axis_contrast_curvse_mag.tick_params(
    axis='both', which='major', labelsize=14)

axis_contrast_curvse_mag.set_title(
    r"$5 \sigma_{\mathcal{N}}$ Contrast Curves",
    fontsize=18, fontweight="bold", y=1.1)

# --------------------------- Legend -----------------------
handles, labels = axis_contrast_curvse.\
    get_legend_handles_labels()

leg1 = fig.legend(handles, labels, 
                  bbox_to_anchor=(0.22, -0.08), 
                  fontsize=14, 
                  title="# PCA components",
                  loc='lower left', ncol=8)

_=plt.setp(leg1.get_title(),fontsize=14)

As you can see the number of PCA components used has a large effect on the contrast curves. We can use our analysis reverse to determine the best number of PCA components as a function of separation.

## Best number of PCA components

In [ ]:
plt.figure(figsize=(12, 8))

plt.plot(separations_arcsec, 
         np.array(components)[np.argmin(
             contrast_curves_grid.values, 
             axis=1)],)

plt.title(r"Best number of PCA components",
          fontsize=18, fontweight="bold", y=1.1)

plt.tick_params(axis='both', which='major', labelsize=14)
plt.xlabel("Separation [arcsec]", fontsize=16)
plt.ylabel("Number of PCA components", fontsize=16)

plt.grid()
ax2 = plt.twiny()
ax2.plot(separations_FWHM, 
         np.array(components)[
             np.argmin(contrast_curves_grid.values, axis=1)],)
ax2.set_xlabel("Separation [FWHM]", fontsize=16)
ax2.tick_params(axis='both', which='major', labelsize=14)